In [1]:
from pandas import Series, Timestamp
import pandas as pd
from pandas import DataFrame
import numpy as np
import simplejson as json

In [2]:
def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return pd.DataFrame(lines)

In [52]:
# Load data for town called Ambridge
data_path = '../data/ambridge/'

# Seperate the files 
review_file = 'review.json'
business_file = 'business.json'
user_file = 'user.json'
tip_file = 'tip.json'
checkin_file = 'checkin.json'

reviews = load_jsons(data_path, review_file)
businesses = load_jsons(data_path, business_file)
users = load_jsons(data_path, user_file)
tips = load_jsons(data_path, tip_file)
checkins = load_jsons(data_path, checkin_file)

In [53]:
# Displays the reviews, high star ratings first
highest_stars = (reviews.sort_values(by='stars', ascending=False))
display(personal_stars)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
126,wVKQ1qZssiWnGPJqLnjUBg,gFZpRY97hKb2Oeffjo8dvg,fDQF6ojXgCZsqaqQkrQvdQ,5.0,0,1,0,Best Burger probably EVER! Although get ready ...,2018-07-30 16:15:23
58,iBFirpfG2ONoV8FoaQrRvg,RlSZhAJCkYbyMkvjljeLeQ,fDQF6ojXgCZsqaqQkrQvdQ,5.0,0,0,0,Not knowing exactly what to expect we walked i...,2016-10-03 18:19:43
166,i2kDaW9br-dDgi5x7-GKQQ,Gk5xfuVdsIOHNo_vfoHELg,fDQF6ojXgCZsqaqQkrQvdQ,5.0,2,1,1,A rare occasion for me to visit a restaurant o...,2016-05-25 02:48:11
162,XaFWZiVeVpRCYsvqF2HPtQ,ETOxL8gIBIIASEiyT0DJkg,fDQF6ojXgCZsqaqQkrQvdQ,5.0,0,0,0,This place was amazing the burgers were mouth-...,2018-08-11 20:38:43
142,pVTq7BhsunUxpvVm6Om0XA,S3d7GcXk90qI8IyOmbMT3A,fDQF6ojXgCZsqaqQkrQvdQ,5.0,0,0,0,This place has the best burgers in town! Our s...,2017-08-15 19:59:08
...,...,...,...,...,...,...,...,...,...
43,Hi35lcHVBQ23NDbye_kYEQ,1hkzLE-HBuAlPIbk7Ob9Rw,fDQF6ojXgCZsqaqQkrQvdQ,1.0,0,0,0,"No gluten free buns!!?? Really, even cheap bur...",2017-02-11 21:48:37
191,gEXVHGcbpXoZXFdIof3gQg,NCKuhe47yqcMQ_v-kQPmzA,fDQF6ojXgCZsqaqQkrQvdQ,1.0,0,0,0,I came here with my husband for dinner on our ...,2016-09-04 03:01:21
229,M58oQEaXfOHv2I-GimqEHA,nLrfvJnDAK4C0KMr5PwiGA,fDQF6ojXgCZsqaqQkrQvdQ,1.0,0,0,0,"Wow, this place is bad!!! We grabbed a quick d...",2017-04-18 23:17:11
137,luspYqrc4GxwoUYjwjPTRA,Yo0ZBgaphTWeWETiVT03kQ,fDQF6ojXgCZsqaqQkrQvdQ,1.0,0,0,0,Disappointing; not the same. I have eaten here...,2016-12-29 13:29:22


In [5]:
"""
All these functions were taken and modified from module 3
"""

def extract_categories(businesses):
    categories = businesses.apply(lambda row: pd.Series([row['business_id']] + row['categories'].lower().split(",")), axis=1)
    stack_categories = categories.set_index(0).stack()
    df_stack_categories = stack_categories.to_frame()
    df_stack_categories['business_id'] = stack_categories.index.droplevel(1)
    df_stack_categories.columns = ['category', 'business_id']
    return df_stack_categories.reset_index()[['business_id', 'category']]

def pivot_categories(df):
    """Create a one-hot encoded matrix for categories.
    
    Arguments:
    df -- a dataFrame containing at least the columns 'businessId' and 'category'
    
    Output:
    a matrix containing '0' or '1' in each cell.
    1: the business has the category
    0: the business does not have the category
    """
    return df.pivot_table(index = 'business_id', columns = 'category', aggfunc = 'size', fill_value=0)

def create_similarity_matrix_jaccard(matrix):
    m11 = matrix @ matrix.T
    m10 = pd.DataFrame(matrix.sum(axis = 1).values + np.zeros(m11.shape), index = m11.index, columns = m11.index)
    m01 = m10.T
    return m11/(m01 + m10 - m11)

categories = extract_categories(businesses)
df_utility_categories = pivot_categories(categories)
similarity_matrix_categories = create_similarity_matrix_jaccard(df_utility_categories)

In [21]:
display(pivot_categories(categories))


category,american (new),american (traditional),appliances,appliances & repair,arts & entertainment,auto detailing,auto parts & supplies,automotive,barbeque,bars,...,local services,lounges,mexican,museums,pets,pizza,restaurants,sandwiches,shopping,southern
business_id,,,,,,,,,,,,,,,,,,,,,
-InU2nAbC9AuS-Um2Cowgw,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0EyRe-VkW8gYxxZ7NCQXeQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2_7lYF6P2cYKnSiPhhVyPQ,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3gL18eXylqutlzqb6TmB0w,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4mpSNvmyG89Uqy2ahP4JMQ,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
729grSa1Wsn-hfv7D5uOxg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7xRbOtZUuw7DxOWrHeaanw,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
EOmRHPvzR88a5R2j2uD0cQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
EZ9zSk4ld27LwgfANoF4VA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
display(similarity_matrix_categories.head())

business_id,-InU2nAbC9AuS-Um2Cowgw,0EyRe-VkW8gYxxZ7NCQXeQ,2_7lYF6P2cYKnSiPhhVyPQ,3gL18eXylqutlzqb6TmB0w,4mpSNvmyG89Uqy2ahP4JMQ,729grSa1Wsn-hfv7D5uOxg,7xRbOtZUuw7DxOWrHeaanw,EOmRHPvzR88a5R2j2uD0cQ,EZ9zSk4ld27LwgfANoF4VA,Eu_zPTrNVAXkpdSxf7CJ2w,...,lawYwEXAE-Sq2nf6co7aBg,muFJIZKZwbAfy_pEFKF_pw,pcbVl6ZHDOJHsnetY0rJEQ,rdHO0LkiNe6s3716hPuQXQ,tTDxa8OzmxUkpVifQ-cEWw,t_EiW3FlMnFTHyoeVU79xg,xM8dVGLkYaL94EuAIkjMEA,y3IVqEFHmrkgVKj2x1Ci4w,yyGzYDh0Qa2o8vUGMEjDRA,zBeUDwWx73QTZ34A1l0adQ
business_id,,,,,,,,,,,,,,,,,,,,,
-InU2nAbC9AuS-Um2Cowgw,1.0,0.000000,0.000000,0.0,0.000000,0.20,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.0,0.000000,0.00,0.25,0.0,0.0,0.0,0.00
0EyRe-VkW8gYxxZ7NCQXeQ,0.0,1.000000,0.166667,0.0,0.166667,0.25,0.200000,0.0,0.0,0.200000,...,0.000000,0.25,0.0,0.166667,0.25,0.00,0.0,0.0,0.0,0.25
2_7lYF6P2cYKnSiPhhVyPQ,0.0,0.166667,1.000000,0.0,0.142857,0.00,0.166667,0.0,0.0,0.166667,...,0.166667,0.20,0.0,0.600000,0.20,0.25,0.0,0.0,0.0,0.20
3gL18eXylqutlzqb6TmB0w,0.0,0.000000,0.000000,1.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,...,0.000000,0.00,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.00
4mpSNvmyG89Uqy2ahP4JMQ,0.0,0.166667,0.142857,0.0,1.000000,0.00,0.166667,0.0,0.0,0.166667,...,0.000000,0.20,0.0,0.142857,0.20,0.00,0.0,0.0,0.0,0.20


In [13]:
user_dataframe = pd.DataFrame(users)

display(user_dataframe)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,3FjdHwnoz-vfw6hBBtckqg,Maria,445,2008-03-26 18:35:10,718,193,377,"2008,2009,2014,2015,2016,2017,2018","lgKMgS9KCIo5olvvM5gClw, y6og6ueUObeQNYkzgNjZDg...",31,...,6,0,1,1,15,15,34,34,27,3
1,rCWrxuRC8_pfagpchtHp6A,Darren,1449,2009-02-06 06:45:55,17459,13033,14714,,"GvqIvPeMkchQ8jYhPY7lXg, 8ugd_yB1RecZMCWfSsr3GQ...",250,...,115,120,37,37,758,2123,1408,1408,1151,403
2,9YIEYRTM1-esrPtfTVU4LA,David,29,2014-05-18 22:53:49,63,14,14,"2016,2017","C4fKnSeNkD904ZeQLr5xVw, rVAiIWafD-4UuLLnjLzUMA...",4,...,0,0,0,0,1,2,1,1,1,0
3,_pLmakEC1U8UPpxCUk2ZAg,Erin,12,2010-10-21 23:00:28,8,2,2,,"wDF8cwQOjn87BjjPDr6vTw, uB4NDnKvS-O07cQ5pq_eCA...",0,...,0,0,0,0,0,0,1,1,0,0
4,ZnIaEvJwrxlBUjimIpx2Qw,Kat,240,2010-08-12 04:40:22,220,67,97,"2016,2017,2018","aPEj8geDZjVNT53wvsWEUQ, getX31OKf2IZ5QvpruTNtQ...",4,...,0,0,0,0,2,1,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,lL9Sppyk-mzNGJ5Uloz5-g,Mark,4,2013-03-15 22:57:48,4,0,0,,"XBMFEOxsul_HALa34wEY8w, H0QzvFRXIIpTxj6nWCH_GQ...",0,...,0,0,0,0,0,0,0,0,0,0
277,fM8rythOmJGL-IRsiYdZqg,Alfred,5,2017-03-09 18:05:53,0,0,1,,"boXAA22H2Z56XMRGf-fj7g, WlDARpCLkOBGqa20X8b3hA...",2,...,0,0,0,0,0,0,0,0,0,0
278,IjiXC9ekXkP1I9kdLWogEw,Jeff,2,2015-09-19 23:49:11,0,0,0,,None,0,...,0,0,0,0,0,0,0,0,0,0
279,01vXmAJKQ41PVi2wgYDiPA,Ca,5,2016-02-19 19:06:56,5,1,0,,None,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def predict_ratings(similarity, utility, to_predict):
    """Predicts the predicted rating for the input test data.
    
    Arguments:
    similarity -- a dataFrame that describes the similarity between items
    utility    -- a dataFrame that contains a rating for each user (columns) and each movie (rows). 
                  If a user did not rate an item the value np.nan is assumed. 
    to_predict -- A dataFrame containing at least the columns movieId and userId for which to do the predictions
    """
    # copy input (don't overwrite)
    ratings_test_c = to_predict.copy()
    # apply prediction to each row
    ratings_test_c['predicted rating'] = to_predict.apply(lambda row: predict_ids(similarity, utility, row['categories'], row['business_id']), axis=1)
    return ratings_test_c


def predict_ids(similarity, utility, userId, itemId):
    # select right series from matrices and compute
    if userId in utility.columns and itemId in similarity.index:
        return predict_vectors(utility.loc[:,userId], similarity[itemId])
    return 0

In [11]:
prediction = predict_ratings(similarity_matrix_categories, businesses, businesses[['name', 'business_id', 'categories']])
display(prediction)

,name,business_id,categories,predicted rating
0,Ice Cream Therapy,dJ0R-XT78LUQeNHQkD-G9g,"Food, Ice Cream & Frozen Yogurt",0
1,Action Tire Company,3gL18eXylqutlzqb6TmB0w,"Auto Repair, Smog Check Stations, Tires, Autom...",0
2,Nelia's Smokehouse,Q_0eGl-aElqHKukHvmLdwA,"Filipino, Food, Restaurants, Barbeque, Smokehouse",0
3,K & N Restaurant,Eu_zPTrNVAXkpdSxf7CJ2w,"Breakfast & Brunch, Restaurants, American (Tra...",0
4,Heritage Floral Shoppe,Yjf0i2J9q52dYIT8UVGT3g,"Florists, Flowers & Gifts, Shopping",0
5,Off the Hook Exotics,y3IVqEFHmrkgVKj2x1Ci4w,"Pets, Pet Stores",0
6,Pizza House,729grSa1Wsn-hfv7D5uOxg,"Restaurants, Pizza",0
7,Firehouse Lounge,rdHO0LkiNe6s3716hPuQXQ,"Lounges, Nightlife, Restaurants, Bars",0
8,Pucktastic Auto Detailing,iJhb_2JL1uIbIRYUl41uVg,"Car Wash, Automotive, Auto Detailing",0
9,Ambridge Wholesale Tire,NHhzHVKVizOvA61AyN-dSw,"Auto Repair, Automotive, Tires, Auto Parts & S...",0
